<a href="https://colab.research.google.com/github/arpagon/podcast-coffeebreak/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detect GPU

In [ ]:
!nvidia-smi -L

# Clone Repo

In [ ]:
!git clone https://github.com/arpagon/podcast-coffeebreak.git

# Install mambaforge

In [ ]:
! wget -O miniconda.sh https://repo.anaconda.com/miniconda/Miniconda3-py37_4.10.3-Linux-x86_64.sh
! chmod +x miniconda.sh
! bash ./miniconda.sh -b -f -p /usr/local
! rm miniconda.sh
! conda config --add channels conda-forge
! conda install -y mamba
! mamba update -qy --all
! mamba clean -qafy
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')